<a href="https://colab.research.google.com/github/Sepren/Deep-Learning/blob/main/Lab_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 17.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 15.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 13.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 15.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━╸━━━━━━━━━━━━━━━━ 391.4/664.8 MB 116.1 MB/s eta 0:00:03

In [ ]:
import numpy as np
import pandas as pd
import torch
import os
import yaml
import cv2
import json
import torch

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import json
import pandas as pd

# Загрузка аннотаций
annotations_path = '/content/drive/MyDrive/Детекция/wind-turbine-damage-challenges/Wind Turbine damage/train/_annotations.coco.json'
with open(annotations_path) as f:
    coco_data = json.load(f)


categories = {cat['id']: cat['name'] for cat in coco_data['categories']}
images = {img['id']: img['file_name'] for img in coco_data['images']}


target_category_ids = {0, 1, 2, 3, 4, 5}
unique_image_ids = set()


data = []
for ann in coco_data['annotations']:
    category_id = ann['category_id']
    if category_id in target_category_ids:
        image_id = ann['image_id']
        file_name = images.get(image_id, f"unknown_{image_id}")
        unique_image_ids.add(file_name)

        x, y, w, h = ann['bbox']
        obj_record = [categories[category_id], x, y, w, h]

        found = False
        for entry in data:
            if entry['image_id'] == file_name:
                entry['objects'].append(obj_record)
                found = True
                break

        if not found:
            data.append({'image_id': file_name, 'objects': [obj_record]})


df = pd.DataFrame(data)
df['objects'] = df['objects'].apply(lambda x: str(x).replace("'", '"'))

# Вывод количества уникальных фотографий
num_unique_images = len(unique_image_ids)
print(f"Количество уникальных фотографий с интересующими категориями: {num_unique_images}")

Количество уникальных фотографий с интересующими категориями: 754


In [ ]:
annotations_path = '/content/drive/MyDrive/Детекция/wind-turbine-damage-challenges/Wind Turbine damage/train/_annotations.coco.json'
with open(annotations_path) as f:
    coco_data = json.load(f)

unique_categories = {cat['id']: cat['name'] for cat in coco_data['categories']}
num_categories = len(unique_categories)

print(f"Количество уникальных категорий: {num_categories}")
print("Список категорий:")
for category_id, category_name in unique_categories.items():
    print(f"ID: {category_id}, Название: {category_name}")

Количество уникальных категорий: 6
Список категорий:
ID: 0, Название: damage
ID: 1, Название: corrosion
ID: 2, Название: lightning
ID: 3, Название: lightning receptor
ID: 4, Название: missing teeth
ID: 5, Название: patch


# Колво фоток по категориям на train

In [ ]:
annotations_path = '/content/drive/MyDrive/Детекция/wind-turbine-damage-challenges/Wind Turbine damage/train/_annotations.coco.json'
with open(annotations_path) as f:
    coco_data = json.load(f)

categories = {cat['id']: cat['name'] for cat in coco_data['categories']}
images = {img['id']: img['file_name'] for img in coco_data['images']}

category_image_counts = {cat_id: set() for cat_id in categories.keys()}

for ann in coco_data['annotations']:
    category_id = ann['category_id']
    image_id = ann['image_id']
    file_name = images.get(image_id, f"unknown_{image_id}")

    if category_id in category_image_counts:
        category_image_counts[category_id].add(file_name)

print("Количество уникальных фотографий по категориям:")
for cat_id, file_names in category_image_counts.items():
    num_unique_images = len(file_names)
    category_name = categories[cat_id]
    print(f"ID: {cat_id}, Название: {category_name}, Количество фотографий: {num_unique_images}")

Количество уникальных фотографий по категориям:
ID: 0, Название: damage, Количество фотографий: 0
ID: 1, Название: corrosion, Количество фотографий: 534
ID: 2, Название: lightning, Количество фотографий: 31
ID: 3, Название: lightning receptor, Количество фотографий: 82
ID: 4, Название: missing teeth, Количество фотографий: 372
ID: 5, Название: patch, Количество фотографий: 76


# Колво фоток по категориям на валидации

In [ ]:
annotations_path = '/content/drive/MyDrive/Детекция/wind-turbine-damage-challenges/Wind Turbine damage/valid/_annotations.coco.json'
with open(annotations_path) as f:
    coco_data = json.load(f)

categories = {cat['id']: cat['name'] for cat in coco_data['categories']}
images = {img['id']: img['file_name'] for img in coco_data['images']}

category_image_counts = {cat_id: set() for cat_id in categories.keys()}

for ann in coco_data['annotations']:
    category_id = ann['category_id']
    image_id = ann['image_id']
    file_name = images.get(image_id, f"unknown_{image_id}")

    if category_id in category_image_counts:
        category_image_counts[category_id].add(file_name)

print("Количество уникальных фотографий по категориям:")
for cat_id, file_names in category_image_counts.items():
    num_unique_images = len(file_names)
    category_name = categories[cat_id]
    print(f"ID: {cat_id}, Название: {category_name}, Количество фотографий: {num_unique_images}")

Количество уникальных фотографий по категориям:
ID: 0, Название: damage, Количество фотографий: 0
ID: 1, Название: corrosion, Количество фотографий: 134
ID: 2, Название: lightning, Количество фотографий: 9
ID: 3, Название: lightning receptor, Количество фотографий: 10
ID: 4, Название: missing teeth, Количество фотографий: 81
ID: 5, Название: patch, Количество фотографий: 16


# Аугментация

In [ ]:
import albumentations as A
from tqdm import tqdm
import ast

In [ ]:
annotations_path = '/content/drive/MyDrive/Детекция/wind-turbine-damage-challenges/Wind Turbine damage/train/_annotations.coco.json'

images_folder = '/content/drive/MyDrive/Детекция/wind-turbine-damage-challenges/Wind Turbine damage/train/'
output_folder = '/content/drive/MyDrive/Детекция/wind-turbine-damage-challenges/Wind Turbine damage/output/'

os.makedirs(output_folder, exist_ok=True)

with open(annotations_path) as f:
    coco_data = json.load(f)

categories = {cat['id']: cat['name'] for cat in coco_data['categories']}
images = {img['id']: img['file_name'] for img in coco_data['images']}

data = []
for ann in coco_data['annotations']:
    image_id = ann['image_id']
    file_name = images.get(image_id, f"unknown_{image_id}")
    category = categories.get(ann['category_id'], "unknown")
    x, y, w, h = ann['bbox']
    obj_record = [category, x, y, w, h]

    found = False
    for entry in data:
        if entry['image_id'] == file_name:
            entry['objects'].append(obj_record)
            found = True
            break

    if not found:
        data.append({'image_id': file_name, 'objects': [obj_record]})

df = pd.DataFrame(data)
df['objects'] = df['objects'].apply(lambda x: str(x).replace("'", '"'))

transform = A.Compose([
    A.HorizontalFlip(p=0.5),
    A.VerticalFlip(p=0.3),
    A.RandomRotate90(p=0.3),
    A.ShiftScaleRotate(shift_limit=0.05, scale_limit=0.1, rotate_limit=15, p=0.5),
    A.RandomBrightnessContrast(brightness_limit=0.2, contrast_limit=0.2, p=0.3),
    A.HueSaturationValue(hue_shift_limit=10, sat_shift_limit=20, val_shift_limit=10, p=0.2),
], bbox_params=A.BboxParams(format='coco', label_fields=['category_names']))

def augment_image(row, augmentations_per_image=4):
    new_rows = []
    image_path = os.path.join(images_folder, row['image_id'])
    image = cv2.imread(image_path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    objects = ast.literal_eval(row['objects'])
    bboxes = [obj[1:] for obj in objects]
    category_names = [obj[0] for obj in objects]

    for i in range(augmentations_per_image):
        try:
            transformed = transform(
                image=image,
                bboxes=bboxes,
                category_names=category_names
            )

            new_filename = f"aug{i}_{row['image_id']}"
            cv2.imwrite(os.path.join(output_folder, new_filename),
                        cv2.cvtColor(transformed['image'], cv2.COLOR_RGB2BGR))

            new_objects = [
                [cat, *bbox]
                for bbox, cat in zip(transformed['bboxes'], category_names)
            ]

            new_rows.append({
                'image_id': new_filename,
                'objects': str(new_objects).replace("'", '"')
            })
        except Exception as e:
            print(f"Ошибка при аугментации {row['image_id']}: {str(e)}")

    return new_rows

augmented_data = []
for _, row in tqdm(df.iterrows(), total=len(df)):
    augmented_data.append(row.to_dict())
    augmented_data.extend(augment_image(row))

augmented_df = pd.DataFrame(augmented_data)

augmented_df.to_csv(os.path.join(output_folder, 'augmented_annotations.csv'), index=False)

print(f"Оригинальный датасет: {len(df)} изображений")
print(f"Аугментированный датасет: {len(augmented_df)} изображений")
print("\nПример аугментированных данных:")
print(augmented_df.head())

/usr/local/lib/python3.11/dist-packages/albumentations/core/validation.py:114: UserWarning: ShiftScaleRotate is a special case of Affine transform. Please use Affine transform instead.
  original_init(self, **validated_kwargs)
  6%|▌         | 45/754 [01:56<28:04,  2.38s/it]

Ошибка при аугментации 20240120_162942_jpg.rf.0270798612537561a390df5dbdd72a98.jpg: Expected y_max for bbox [0.6520722 0.967246  0.6862934 1.0001336 0.       ] to be in the range [0.0, 1.0], got 1.0001336336135864.
Ошибка при аугментации 20240120_162942_jpg.rf.0270798612537561a390df5dbdd72a98.jpg: Expected y_max for bbox [0.6520722 0.967246  0.6862934 1.0001336 0.       ] to be in the range [0.0, 1.0], got 1.0001336336135864.
Ошибка при аугментации 20240120_162942_jpg.rf.0270798612537561a390df5dbdd72a98.jpg: Expected y_max for bbox [0.6520722 0.967246  0.6862934 1.0001336 0.       ] to be in the range [0.0, 1.0], got 1.0001336336135864.
Ошибка при аугментации 20240120_162942_jpg.rf.0270798612537561a390df5dbdd72a98.jpg: Expected y_max for bbox [0.6520722 0.967246  0.6862934 1.0001336 0.       ] to be in the range [0.0, 1.0], got 1.0001336336135864.


 20%|██        | 152/754 [06:59<16:11,  1.61s/it]

Ошибка при аугментации 20240120_162912_jpg.rf.29699b9c2c3bc8888f76923fb517f311.jpg: Expected y_max for bbox [0.7616979 0.9906417 0.8364572 1.0000601 0.       ] to be in the range [0.0, 1.0], got 1.0000600814819336.
Ошибка при аугментации 20240120_162912_jpg.rf.29699b9c2c3bc8888f76923fb517f311.jpg: Expected y_max for bbox [0.7616979 0.9906417 0.8364572 1.0000601 0.       ] to be in the range [0.0, 1.0], got 1.0000600814819336.
Ошибка при аугментации 20240120_162912_jpg.rf.29699b9c2c3bc8888f76923fb517f311.jpg: Expected y_max for bbox [0.7616979 0.9906417 0.8364572 1.0000601 0.       ] to be in the range [0.0, 1.0], got 1.0000600814819336.
Ошибка при аугментации 20240120_162912_jpg.rf.29699b9c2c3bc8888f76923fb517f311.jpg: Expected y_max for bbox [0.7616979 0.9906417 0.8364572 1.0000601 0.       ] to be in the range [0.0, 1.0], got 1.0000600814819336.


 22%|██▏       | 168/754 [07:34<20:02,  2.05s/it]

...

 89%|████████▉ | 673/754 [35:20<04:16,  3.16s/it]

Ошибка при аугментации DJI_0276_JPG.rf.9ab500a2b99c5e877759996d7f0a7777.jpg: Expected y_max for bbox [0.24185605 0.9754209  0.2730341  1.0000504  0.        ] to be in the range [0.0, 1.0], got 1.00005042552948.
Ошибка при аугментации DJI_0276_JPG.rf.9ab500a2b99c5e877759996d7f0a7777.jpg: Expected y_max for bbox [0.24185605 0.9754209  0.2730341  1.0000504  0.        ] to be in the range [0.0, 1.0], got 1.00005042552948.
Ошибка при аугментации DJI_0276_JPG.rf.9ab500a2b99c5e877759996d7f0a7777.jpg: Expected y_max for bbox [0.24185605 0.9754209  0.2730341  1.0000504  0.        ] to be in the range [0.0, 1.0], got 1.00005042552948.
Ошибка при аугментации DJI_0276_JPG.rf.9ab500a2b99c5e877759996d7f0a7777.jpg: Expected y_max for bbox [0.24185605 0.9754209  0.2730341  1.0000504  0.        ] to be in the range [0.0, 1.0], got 1.00005042552948.


 97%|█████████▋| 728/754 [38:45<01:02,  2.41s/it]

Ошибка при аугментации DJI_0181_JPG.rf.4ce43776caf50c48ad56c02fef6b3005.jpg: Expected y_max for bbox [0.5299242  0.97946125 0.5345549  1.0000639  0.        ] to be in the range [0.0, 1.0], got 1.0000638961791992.
Ошибка при аугментации DJI_0181_JPG.rf.4ce43776caf50c48ad56c02fef6b3005.jpg: Expected y_max for bbox [0.5299242  0.97946125 0.5345549  1.0000639  0.        ] to be in the range [0.0, 1.0], got 1.0000638961791992.
Ошибка при аугментации DJI_0181_JPG.rf.4ce43776caf50c48ad56c02fef6b3005.jpg: Expected y_max for bbox [0.5299242  0.97946125 0.5345549  1.0000639  0.        ] to be in the range [0.0, 1.0], got 1.0000638961791992.
Ошибка при аугментации DJI_0181_JPG.rf.4ce43776caf50c48ad56c02fef6b3005.jpg: Expected y_max for bbox [0.5299242  0.97946125 0.5345549  1.0000639  0.        ] to be in the range [0.0, 1.0], got 1.0000638961791992.


100%|██████████| 754/754 [39:41<00:00,  3.16s/it]

In [ ]:
#  данные после аугментации
category_counts = {
    "corrosion": 0,
    "lightning": 0,
    "lightning receptor": 0,
    "missing teeth": 0,
    "patch": 0
}

for obj_list in augmented_df['objects']:
    objects = ast.literal_eval(obj_list)

    for obj in objects:
        category_name = obj[0]
        if category_name in category_counts:
            category_counts[category_name] += 1

# Выводим результаты
for category_id, category_name in zip(range(1, len(category_counts) + 1), category_counts.keys()):
    print(f"ID: {category_id}, Название: {category_name}, Количество: {category_counts[category_name]}")

ID: 1, Название: corrosion, Количество: 33468
ID: 2, Название: lightning, Количество: 155
ID: 3, Название: lightning receptor, Количество: 463
ID: 4, Название: missing teeth, Количество: 7957
ID: 5, Название: patch, Количество: 1166


In [ ]:
!cp -r "/content/drive/MyDrive/CNN/Wind Turbine damage/yolo_wind_turbine_backup" "/content/"

cp: cannot stat '/content/drive/MyDrive/CNN/Wind Turbine damage/yolo_wind_turbine_backup': No such file or directory


In [ ]:
augmented_df = pd.read_csv('/content/drive/MyDrive/Wind Turbine damage/ augmented_df_full.csv')
df_valid = pd.read_csv('/content/drive/MyDrive/Wind Turbine damage/ df_valid.csv')

In [ ]:
base_path = '/content/drive/MyDrive/Детекция/wind-turbine-damage-challenges/Wind Turbine damage/yolo_wind'

In [ ]:
# Создаем YAML файл конфигурации
yaml_content = {
    'path': base_path,
    'train': 'images/train',
    'val': 'images/val',
    'names': ['corrosion', 'lightning', 'lightning receptor', 'missing teeth', 'patch']
}

with open(os.path.join(base_path, 'wind_turbine.yaml'), 'w') as f:
    yaml.dump(yaml_content, f)

print("Подготовка данных завершена!")

Подготовка данных завершена!


In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"Используемое устройство: {device}")

Используемое устройство: cpu


In [ ]:
! pip install ultralytics

In [ ]:
import os
from pathlib import Path
import shutil
import json

# Пути с учётом пробелов
coco_train_path = '/content/drive/MyDrive/Детекция/wind-turbine-damage-challenges/Wind Turbine damage/train/_annotations.coco.json'
coco_val_path = '/content/drive/MyDrive/Детекция/wind-turbine-damage-challenges/Wind Turbine damage/valid/_annotations.coco.json'
yolo_base_path = '/content/drive/MyDrive/Детекция/wind-turbine-damage-challenges/Wind Turbine damage/yolo_wind'
images_train_path = os.path.join(yolo_base_path, 'images/train')
images_val_path = os.path.join(yolo_base_path, 'images/val')
labels_train_path = os.path.join(yolo_base_path, 'labels/train')
labels_val_path = os.path.join(yolo_base_path, 'labels/val')
source_train_path = '/content/drive/MyDrive/Детекция/wind-turbine-damage-challenges/Wind Turbine damage/train'
source_val_path = '/content/drive/MyDrive/Детекция/wind-turbine-damage-challenges/Wind Turbine damage/valid'

# Создание папок
Path(images_train_path).mkdir(parents=True, exist_ok=True)
Path(images_val_path).mkdir(parents=True, exist_ok=True)
Path(labels_train_path).mkdir(parents=True, exist_ok=True)
Path(labels_val_path).mkdir(parents=True, exist_ok=True)

# Классы
class_names = ['corrosion', 'lightning', 'lightning receptor', 'missing teeth', 'patch']
class_map = {name: idx for idx, name in enumerate(class_names)}

def coco_to_yolo(coco_json_path, images_dir, labels_dir, source_dir):
    if not os.path.exists(coco_json_path):
        print(f"Файл {coco_json_path} не найден!")
        return
    with open(coco_json_path, 'r') as f:
        coco_data = json.load(f)

    image_id_to_filename = {img['id']: img['file_name'] for img in coco_data['images']}
    annotations = coco_data['annotations']
    categories = {cat['id']: cat['name'] for cat in coco_data['categories']}
    images_copied = 0
    labels_created = 0

    for ann in annotations:
        image_id = ann['image_id']
        filename = image_id_to_filename.get(image_id)
        if not filename:
            print(f"Изображение с ID {image_id} не найдено в аннотациях!")
            continue
        img_width = next(img['width'] for img in coco_data['images'] if img['id'] == image_id)
        img_height = next(img['height'] for img in coco_data['images'] if img['id'] == image_id)

        bbox = ann['bbox']
        x_min, y_min, width, height = bbox
        x_center = (x_min + width / 2) / img_width
        y_center = (y_min + height / 2) / img_height
        normalized_width = width / img_width
        normalized_height = height / img_height

        category_id = ann['category_id']
        class_id = class_map.get(categories.get(category_id), -1)
        if class_id == -1:
            print(f"Категория с ID {category_id} не найдена в class_map!")
            continue

        src_img_path = os.path.join(source_dir, filename)
        if os.path.exists(src_img_path):
            dst_img_path = os.path.join(images_dir, filename)
            Path(dst_img_path).parent.mkdir(parents=True, exist_ok=True)
            shutil.copy(src_img_path, dst_img_path)
            images_copied += 1
        else:
            print(f"Изображение {filename} не найдено в {source_dir}!")

        label_filename = os.path.splitext(filename)[0] + '.txt'
        label_path = os.path.join(labels_dir, label_filename)
        with open(label_path, 'a') as f:
            f.write(f"{class_id} {x_center} {y_center} {normalized_width} {normalized_height}\n")
            labels_created += 1

    print(f"Скопировано {images_copied} изображений, создано {labels_created} аннотаций в {labels_dir}")

# Выполнение конвертации
coco_to_yolo(coco_train_path, images_train_path, labels_train_path, source_train_path)
coco_to_yolo(coco_val_path, images_val_path, labels_val_path, source_val_path)

print("Конвертация завершена. Проверьте папки images и labels.")

Скопировано 9150 изображений, создано 9150 аннотаций в /content/drive/MyDrive/Детекция/wind-turbine-damage-challenges/Wind Turbine damage/yolo_wind/labels/train
Скопировано 2619 изображений, создано 2619 аннотаций в /content/drive/MyDrive/Детекция/wind-turbine-damage-challenges/Wind Turbine damage/yolo_wind/labels/val
Конвертация завершена. Проверьте папки images и labels.


In [ ]:
from ultralytics import YOLO

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


In [ ]:
if torch.cuda.is_available():
    device = 'cuda'
    device_id = '0'
    print(f"Используется GPU: {torch.cuda.get_device_name(0)}")
else:
    device = 'cpu'
    device_id = 'cpu'
    print("Используется CPU")

base_path = '/content/drive/MyDrive/Детекция/wind-turbine-damage-challenges/Wind Turbine damage/yolo_wind'
model = YOLO('yolov8n.pt')

try:
    results = model.train(
        data=os.path.join(base_path, 'wind_turbine.yaml'),
        epochs=15,          # Уменьшено с 30 до 15
        batch=16,           # Уменьшено с 64 до 16
        imgsz=416,          # Уменьшено с 640 до 416
        device=device_id,
        workers=4           # Уменьшено с 8 до 4
    )
except Exception as e:
    print(f"Ошибка при обучении: {e}")

Используется CPU
Ultralytics 8.3.153 🚀 Python-3.11.13 torch-2.6.0+cu124 CPU (Intel Xeon 2.20GHz)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/content/drive/MyDrive/Детекция/wind-turbine-damage-challenges/Wind Turbine damage/yolo_wind/wind_turbine.yaml, degrees=0.0, deterministic=True, device=cpu, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=15, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=416, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8n.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=train2, nbs=64, nms=False, opset=None, optimize=False, opt

train: Scanning /content/drive/MyDrive/Детекция/wind-turbine-damage-challenges/Wind Turbine damage/yolo_wind/labels/train... 94 images, 0 backgrounds, 0 corrupt:  12%|█▏        | 94/754 [00:04<00:09, 70.11it/s]/usr/local/lib/python3.11/dist-packages/PIL/Image.py:3442: DecompressionBombWarning: Image size (149817600 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  warnings.warn(
train: Scanning /content/drive/MyDrive/Детекция/wind-turbine-damage-challenges/Wind Turbine damage/yolo_wind/labels/train... 754 images, 0 backgrounds, 0 corrupt: 100%|██████████| 754/754 [00:11<00:00, 66.39it/s]


train: New cache created: /content/drive/MyDrive/Детекция/wind-turbine-damage-challenges/Wind Turbine damage/yolo_wind/labels/train.cache
albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01, method='weighted_average', num_output_channels=3), CLAHE(p=0.01, clip_limit=(1.0, 4.0), tile_grid_size=(8, 8))
val: Fast image access ✅ (ping: 9.6±20.3 ms, read: 83.8±151.8 MB/s, size: 1287.3 KB)


val: Scanning /content/drive/MyDrive/Детекция/wind-turbine-damage-challenges/Wind Turbine damage/yolo_wind/labels/val... 185 images, 0 backgrounds, 0 corrupt: 100%|██████████| 185/185 [00:01<00:00, 92.87it/s]


val: New cache created: /content/drive/MyDrive/Детекция/wind-turbine-damage-challenges/Wind Turbine damage/yolo_wind/labels/val.cache
Plotting labels to runs/detect/train2/labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.001111, momentum=0.9) with parameter groups 57 weight(decay=0.0), 64 weight(decay=0.0005), 63 bias(decay=0.0)
Image sizes 416 train, 416 val
Using 0 dataloader workers
Logging results to runs/detect/train2
Starting training for 15 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


...

      15/15         0G      3.237      2.591      1.041          7        416: 100%|██████████| 48/48 [05:34<00:00,  6.96s/it]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 6/6 [00:40<00:00,  6.72s/it]

                   all        185       2619      0.292     0.0756      0.104     0.0412



15 epochs completed in 1.593 hours.
Optimizer stripped from runs/detect/train2/weights/last.pt, 6.2MB
Optimizer stripped from runs/detect/train2/weights/best.pt, 6.2MB

Validating runs/detect/train2/weights/best.pt...
Ultralytics 8.3.153 🚀 Python-3.11.13 torch-2.6.0+cu124 CPU (Intel Xeon 2.20GHz)
Model summary (fused): 72 layers, 3,006,623 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 6/6 [00:30<00:00,  5.11s/it]


                   all        185       2619      0.305     0.0797      0.131     0.0517
             corrosion        134       2150      0.189     0.0484     0.0379     0.0154
             lightning          9          9        0.3      0.111      0.315      0.136
    lightning receptor         10         12          0          0          0          0
         missing teeth         81        391      0.152      0.046     0.0224    0.00491
                 patch         16         57      0.885      0.193      0.281      0.102
Speed: 0.9ms preprocess, 94.9ms inference, 0.0ms loss, 0.6ms postprocess per image
Results saved to runs/detect/train2


# Предсказание

In [ ]:
best_model = YOLO(os.path.join('runs', 'detect', 'train2', 'weights', 'best.pt'))

In [ ]:

test_folder = '/content/drive/MyDrive/Детекция/wind-turbine-damage-challenges/Wind Turbine damage/test'

results = []

for img_name in os.listdir(test_folder):
    if not img_name.lower().endswith(('.png', '.jpg', '.jpeg')):
        continue

    img_path = os.path.join(test_folder, img_name)
    img = cv2.imread(img_path)
    if img is None:
        continue

    predictions = best_model.predict(img, conf=0.25)

    img_objects = []
    for pred in predictions:
        for box in pred.boxes:
            class_id = int(box.cls)
            class_name = best_model.names[class_id]
            conf = float(box.conf)
            x1, y1, x2, y2 = box.xyxy[0].tolist()
            w = x2 - x1
            h = y2 - y1

            img_objects.append([class_name, x1, y1, w, h, conf])

    results.append({
        "image_id": img_name,
        "objects": img_objects
    })

df_results = pd.DataFrame(results)


0: 416x416 (no detections), 146.1ms
Speed: 4.5ms preprocess, 146.1ms inference, 1.1ms postprocess per image at shape (1, 3, 416, 416)

0: 416x416 (no detections), 183.3ms
Speed: 4.1ms preprocess, 183.3ms inference, 1.5ms postprocess per image at shape (1, 3, 416, 416)

0: 416x416 (no detections), 172.9ms
Speed: 3.7ms preprocess, 172.9ms inference, 1.1ms postprocess per image at shape (1, 3, 416, 416)

0: 416x416 (no detections), 161.6ms
Speed: 4.6ms preprocess, 161.6ms inference, 1.0ms postprocess per image at shape (1, 3, 416, 416)

0: 416x416 (no detections), 187.2ms
Speed: 4.4ms preprocess, 187.2ms inference, 1.2ms postprocess per image at shape (1, 3, 416, 416)

0: 416x416 (no detections), 181.6ms
Speed: 5.2ms preprocess, 181.6ms inference, 1.0ms postprocess per image at shape (1, 3, 416, 416)

0: 416x416 (no detections), 178.7ms
Speed: 4.8ms preprocess, 178.7ms inference, 1.0ms postprocess per image at shape (1, 3, 416, 416)

0: 416x416 (no detections), 171.7ms
Speed: 3.6ms prepr

In [ ]:
df_results

,image_id,objects
0,20240120_153658_jpg.rf.66dc04dc4ee8194ccc19397...,[]
1,20240120_153628_jpg.rf.67030e3c4592587d26d7762...,[]
2,20240120_150729_jpg.rf.5925f54483de8cc52f75fdd...,[]
3,20240120_153748-0-_jpg.rf.6c50a8089bf3ccd32bb6...,[]
4,20240120_153711_jpg.rf.3a50aa9daa881abe4296c7f...,[]
...,...,...
146,DJI_0989_JPG.rf.c88276836792084a0205e542b79fb9...,[]
147,DJI_0997_JPG.rf.9f5f2b599b0927db0d1121a39516f4...,"[[corrosion, 2946.65966796875, 807.39801025390..."
148,DJI_0995_JPG.rf.09f70312420808da47c3a4aaf186ca...,[]
149,DJI_0993_JPG.rf.f33b6663ee70d58a17f1b41a031a28...,[]
